In [3]:
library(magrittr)
library(caret)
library(cattonum)

train=read.csv('C:\\Users\\coco40725\\Documents\\GitHub\\2nd-ML100Days\\data\\titanic_train.csv')
train=train[,-c(1,4,9)]

##extract categorical variable
col.type=rep(0,9)
for (i in 1:9) {
  col.type[i]=class(train[,i])
}
col.factor=which(col.type=='factor')



##only keep categorical column
train.cate=train[,c(col.factor,1)]

## covert NA into a level
for (j in 1:3) {
  train.cate[,j] %<>% addNA(.)
  
}

## fill somethong in empty cell
for (d in 1:3) {
  levels(train.cate[,d])[levels(train.cate[,d])=='']='empty'
}

## linear regression+ lable coding
train.cate.label=train.cate
for (k in 1:3) {
  train.cate.label[,k] %<>% as.numeric(.)
}


t11=Sys.time()
train_control <- trainControl(method="cv", number=5)
cv1= train(as.factor(Survived)~., data=train.cate.label, family=binomial,trControl=train_control, method="glm")
t12=Sys.time()
time1=difftime(t11,t12,units='secs')
racc1=cv1$results[2]
rkappa1=cv1$results[3]

# mean coding + regresison
t11=Sys.time()
train.mean=catto_mean(train.cate,c('Sex','Embarked','Cabin'),response=Survived)
cv1.mean= train(as.factor(Survived)~., data=train.mean, family=binomial,trControl=train_control, method="glm")
racc1.mean=cv1.mean$results[2]
rkappa1.mean=cv1.mean$results[3]
t12=Sys.time()
time1.mean=difftime(t11,t12,units='secs')



## gboost + lable coding
t11=Sys.time()
xgb.fit1=train(as.factor(Survived)~., data=train.cate.label, trControl=train_control, method="xgbTree")
gb.acc1=mean(xgb.fit1$results$Accuracy)
gb.kappa1=mean(xgb.fit1$results$Kappa)
t12=Sys.time()
gb.mse1=mean(unlist(xgb.fit1$evaluation_log[,4]))
time.gb1=difftime(t11,t12,units='secs')

# mean coding + xgboost
t11=Sys.time()
xgb.fit1.mean=train(as.factor(Survived)~., data=train.mean, trControl=train_control, method="xgbTree")
t12=Sys.time()
gb.acc2=mean(xgb.fit1.mean$results$Accuracy)
gb.kappa2=mean(xgb.fit1.mean$results$Kappa)
time.gb1.mean=difftime(t11,t12,units='secs')


result.fin=data.frame(type=c('reg+label','reg+mean','xgb+label','xgb+mean'),
                      acc=as.numeric(c(racc1,racc1.mean,gb.acc1,gb.acc2)),
                      kappa=as.numeric(c(rkappa1,rkappa1.mean,gb.kappa1,gb.kappa2)),
                      time=-c(time1,time1.mean,time.gb1,time.gb1.mean))
result.fin


ERROR: Error in library(cattonum): there is no package called 'cattonum'
